# I. Hot Spot Model

In [ ]:
#Throughout this document, things that can be changed by the user to investigate different 
#parts of the map are marked with #FINDME

# Import functions
import numpy as np
import matplotlib.pyplot as p
%matplotlib inline

import eigencurves
import eigenmaps
import kmeans
import bin_eigenspectra
import os
import pdb

import gen_lightcurves
import healpy

import colorcet as cc
from colormap2d import generate_map2d
from matplotlib import colorbar, cm
from matplotlib.colors import BoundaryNorm, Normalize
from matplotlib.ticker import FormatStrFormatter, ScalarFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
p.rc('text', usetex=True)
p.rc('font', family='serif')

import run_higher_sph_harm

from importlib import import_module
planet_name = 'HD189733b'
model_name = 'hotspot' #FINDME: name of the model to perform the fit from

## 1. Generate lightcurve using STARRY

In [ ]:
# Get the high-res wavelength grid and pixel spectra (spaxels)
lamhr, spaxels = gen_lightcurves.prep_spectral_hotspot_map()

# Generate the multi-wavelength eclipse lightcurves using starry
time, lam, dlam, lcurves = gen_lightcurves.create_lightcurves_with_starry(lamhr, spaxels, plot_lightcurves = False, plot_diagnostic = False)

# Save as a .npz file in data/input_lightcurves
filename='data/input_lightcurves/'+model_name+'.npz'
np.savez(filename,time=time,wl=lam,dwl=dlam,lightcurve=lcurves)


## 2. Fit eigencurves to lightcurve

In [ ]:
## Results are saved in data/sph_harmonic_coefficients_full_samples
for oneOrd in np.arange(2,6): #which degrees of spherical harmonics you want to create fits for
    run_higher_sph_harm.run_lc_noise_and_fit(norder=oneOrd,usePath=filename)


## 3. Display maps for median, 5th percentile, and 95th percentile from spherical harmonic fit

In [ ]:
def retrieve_map_full_samples(degree=3,dataDir="data/sph_harmonic_coefficients_full_samples/hotspot/"):
    tmp = np.load("{}spherearray_deg_{}.npz".format(dataDir,degree))
    outDictionary = tmp['arr_0'].tolist()
    
    londim = 100
    latdim = 100
    samples = outDictionary['spherical coefficients'] # output from eigencurves
    waves = outDictionary['wavelength (um)']
    
    randomIndices = np.random.randint(0,len(samples),99)
    nRandom = len(randomIndices)
    
    fullMapArray = np.zeros([nRandom,len(waves),londim,latdim])
    for drawInd, draw in enumerate(samples[randomIndices]):
        inputArr = np.zeros([len(waves),samples.shape[1]+1])
        inputArr[:,0] = waves
        inputArr[:,1:] = draw.transpose()
        
        wavelengths, lats, lons, maps = eigenmaps.generate_maps(inputArr,
                                                                N_lon=londim, N_lat=latdim)
        fullMapArray[drawInd,:,:,:] = maps
    
    return fullMapArray, lats, lons, waves
    


In [ ]:
def plot_retrieved_map(fullMapArray,lats,lons,waves,waveInd=3,degree=3):
    percentiles = [5,50,95]
    mapLowMedHigh = np.percentile(fullMapArray,percentiles,axis=0)
    londim = fullMapArray.shape[2]
    
    fig, axArr = p.subplots(1,3,figsize=(22,5))
    for ind,onePercentile in enumerate(percentiles):
        map_day = mapLowMedHigh[ind][waveInd][:,londim//4:-londim//4]
        extent = np.array([np.min(lons),np.max(lons),np.min(lats),np.max(lats)])/2./np.pi*180
        plotData = axArr[ind].imshow(map_day, extent=extent)
        cbar = fig.colorbar(plotData,ax=axArr[ind])
        cbar.set_label('Brightness')
        axArr[ind].set_ylabel('Latitude')
        axArr[ind].set_xlabel('Longitude')
        axArr[ind].set_title("{} %".format(onePercentile))
        #axArr[ind].show()
    
    fig.suptitle('Retrieved group map, n={}, {:.2f}$\mu$m'.format(degree,waves[waveInd]))

def get_map_and_plot(waveInd=3,degree=3,dataDir="data/sph_harmonic_coefficients_full_samples/hotspot/"):
    '''
    Plots spherical harmonic maps at one wavelength for 5th, 50th, and 95th percentile posterior samples
    
    Inputs
    ----------
    waveInd: int
        Index of the wavelength for which a map will be created
    degree: int
        Spherical harmonic degree to draw samples from
    dataDir: str
        Path to the directory containing the spherical harmonic coefficients
        
    Outputs
    -----------
    waves: array
        Wavelengths for the eigenspectra
    '''
    fullMapArray, lats, lons, waves = retrieve_map_full_samples(degree=degree,dataDir=dataDir)
    plot_retrieved_map(fullMapArray,lats,lons,waves,degree=degree,waveInd=waveInd)
    return waves, lats, lons

def all_sph_degrees(waveInd=5):
    for oneDegree in np.arange(2,6):
        get_map_and_plot(waveInd=waveInd,degree=oneDegree)

In [ ]:
dataDir='data/sph_harmonic_coefficients_full_samples/'+model_name+'/'
waves, lats, lons = get_map_and_plot(waveInd=3,degree=3,dataDir=dataDir) #FINDME: using a different waveInd will show maps at a different wavelength; using a different degree will show a fit with a different number of spherical harmonics


## 4. Use K means clustering to find Eigenspectra

In [ ]:
def find_groups(dataDir,ngroups=4,degree=2,
                londim=100, latdim=100,
                trySamples=45):
    """ 
    Find the eigenspectra using k means clustering
    
    Parameters
    ----------
    ngroups: int
        Number of eigenspectra to group results into
    degree: int
        Spherical harmonic degree to draw samples from
    testNum: int
        Test number (ie. lightcurve number 1,2, etc.)
    trySamples: int
        Number of samples to find groups with
        All samples take a long time so this takes a random
        subset of samples from which to draw posteriors
    """
    #samplesDir = "data/sph_harmonic_coefficients_full_samples"
    #dataDir = "{}/eclipse_lightcurve_test{}/".format(samplesDir,testNum)
    tmp = np.load("{}spherearray_deg_{}.npz".format(dataDir,degree))
    outDictionary = tmp['arr_0'].tolist()
    samples = outDictionary['spherical coefficients'] # output from eigencurves

    eigenspectra_draws = []
    kgroup_draws = []

    randomIndices = np.random.randint(0,len(samples),trySamples)
    for draw in samples[randomIndices]:
        ## Re-formatting here into a legacy system
        ## 1st dimension is wavelength
        ## 2nd dimensions is data (0th element = wavelength)
        ##                        (1: elements are spherical harmonic coefficients)
        inputArr = np.zeros([10,samples.shape[1]+1])
        inputArr[:,0] = np.array([2.41,2.59,2.77,2.95,3.13,3.31,3.49,3.67,3.85,4.03])
        inputArr[:,1:] = draw.transpose()

        wavelengths, lats, lons, maps = eigenmaps.generate_maps(inputArr, N_lon=londim, N_lat=latdim)

        kgroups = kmeans.kmeans(maps, ngroups)

        eigenspectra = bin_eigenspectra.bin_eigenspectra(maps, kgroups)

        eigenspectra_draws.append(eigenspectra)
        kgroup_draws.append(kgroups)
    return eigenspectra_draws, kgroup_draws, maps

In [ ]:
degree=3 #FINDME: change this to investigate fits with different numbers of spherical harmonics
ngroups=3 #FINDME: change ngroups to cluster the data into a different number of eigengroups
eigenspectra_draws, kgroup_draws, maps = find_groups(dataDir,degree=degree,trySamples=200,ngroups=3) 


## 5. Plot the Eigenspectra

In [ ]:
# THIS IS THE PART THAT NEEDS TO BE UPDATED FOR THE ERRORS THING
eigenspectra = np.mean(eigenspectra_draws, axis=0)
eigenerrs = np.std(eigenspectra_draws, axis=0)
kgroups = np.mean(kgroup_draws, axis=0)

counter=0
colors=['b','g','orange']
for spec, err in zip(eigenspectra, eigenerrs):
    p.errorbar(waves, spec, err,label=('Group '+np.str(counter)),linewidth=2,marker='.',markersize=10,color=colors[counter])
    counter+=1
p.xlabel('Wavelength (micron)',fontsize=20)
p.ylabel('Fp/Fs',fontsize=20)
p.tick_params(labelsize=20,axis="both",right=True,top=True,width=1.5,length=5)
p.title('Eigenspectra')
p.legend(fontsize=15)
p.show()
#p.savefig('plots/eigenmap_and_spec/'+'hot_spot_spectra_deg3_3groups_error_bars.pdf',bbox_inches='tight')

## 6. Plot Spatial Map of Eigengroups

In [ ]:
full_extent = np.array([np.min(lons),np.max(lons),np.min(lats),np.max(lats)])/np.pi*180
# londim, latdim = np.shape(maps)[1:]

maps_mean = np.average(maps, axis=0)
maps_error = np.std(maps, axis=0)

cmap = cc.cm['isolum']
cmap_grey = cc.cm['linear_grey_10_95_c0']
# norm = Normalize(vmin=np.min(maps_mean), vmax=np.max(maps_mean))

In [ ]:
p.figure(figsize=(10,6.5))
p.title('Eigengroups', fontsize=22)


group_map = generate_map2d(hue_quantity=0.5*kgroups,
                           lightness_quantity=maps_mean,
                           hue_cmap=cmap,
                           scale_min=10,
                           scale_max=90)
p.imshow(group_map, extent=full_extent, interpolation='gaussian')
CS = p.contour(lons/np.pi*180, -lats/np.pi*180, 2*kgroups,
               levels=np.arange(ngroups), colors='k', linestyles=['solid', 'dashed', 'dotted'])

p.clabel(CS, inline=1, fmt='%1.0f', fontsize=12)

p.xlabel(r'Longitude ($^\circ$)', fontsize=16)
p.ylabel(r'Latitude ($^\circ$)', fontsize=16)
p.setp(p.axes().get_xticklabels(), fontsize=16)
p.setp(p.axes().get_yticklabels(), fontsize=16)

cmap_group = cmap
cNorm_group  = Normalize(vmin=0, vmax=ngroups-1)
scalarMap_group = cm.ScalarMappable(norm=cNorm_group, cmap=cmap_group)

cmap_flux = cmap_grey
cNorm_flux  = Normalize(vmin=0, vmax=np.nanmax(maps_mean))
scalarMap_flux = cm.ScalarMappable(norm=cNorm_flux, cmap=cmap_flux)

bounds = np.linspace(-0.5, ngroups-0.5, ngroups+1)
norm_group = BoundaryNorm(bounds, cmap_group.N)

divider = make_axes_locatable(p.axes())
ax2 = divider.append_axes("bottom", size="7.5%", pad=1)
cb = colorbar.ColorbarBase(ax2, cmap=cmap_group, norm=norm_group, spacing="proportional", orientation='horizontal', ticks=np.arange(0, ngroups, 1), boundaries=bounds)
cb.ax.xaxis.set_minor_formatter(FormatStrFormatter('%.1g'))
cb.ax.tick_params(axis='x', labelsize=13)
cb.ax.tick_params(axis='x', direction='inout',  top='off', bottom='off',
                  labeltop='on', labelbottom='off', labelsize=13, pad=-15)
cb.ax.set_title('Group', y=1.35, fontsize=19)

ax3 = divider.append_axes("bottom", size="7.5%", pad=0.75)
cb = colorbar.ColorbarBase(ax3, cmap=cmap_flux, norm=cNorm_flux, orientation='horizontal')
cb.ax.tick_params(axis='x', labelsize=13)
cb.ax.set_title('Flux', y=1.35, fontsize=19)

for filetype in ['png', 'pdf']:
    p.savefig('HUEgroup_LUMflux_fullmap.{}'.format(filetype), dpi=300, bbox_inches='tight')

In [ ]:
p.figure(figsize=(10,6.5))
p.title('Flux', fontsize=22)

group_map = generate_map2d(hue_quantity=(maps_mean-np.min(maps_mean))/np.ptp(maps_mean),
                           lightness_quantity=1-((maps_error-np.min(maps_error))/np.ptp(maps_error)),
                           hue_cmap=cmap,
                           scale_min=10,
                           scale_max=90)
p.imshow(group_map, extent=full_extent, interpolation='gaussian')
CS = p.contour(lons/np.pi*180, -lats/np.pi*180, 2*kgroups,
               levels=np.arange(ngroups), colors='k', linestyles=['solid', 'dashed', 'dotted'])

p.clabel(CS, inline=1, fmt='%1.0f', fontsize=12)

p.xlabel(r'Longitude ($^\circ$)', fontsize=16)
p.ylabel(r'Latitude ($^\circ$)', fontsize=16)
p.setp(p.axes().get_xticklabels(), fontsize=16)
p.setp(p.axes().get_yticklabels(), fontsize=16)

cmap_flux = cmap
cNorm_flux = Normalize(vmin=0, vmax=np.nanmax(maps_mean))
scalarMap_flux = cm.ScalarMappable(norm=cNorm_flux, cmap=cmap_flux)

cmap_stdev = cmap_grey
cNorm_stdev  = Normalize(vmin=0, vmax=np.nanmax(maps_error))
scalarMap_stdev = cm.ScalarMappable(norm=cNorm_stdev, cmap=cmap_stdev)

divider = make_axes_locatable(p.axes())
ax2 = divider.append_axes("bottom", size="7.5%", pad=1)
cb = colorbar.ColorbarBase(ax2, cmap=cmap_flux, norm=cNorm_flux, orientation='horizontal')
cb.ax.tick_params(axis='x', labelsize=13)
cb.ax.set_title('Flux', y=1.35, fontsize=19)

ax3 = divider.append_axes("bottom", size="7.5%", pad=0.75)
cb = colorbar.ColorbarBase(ax3, cmap=cmap_stdev, norm=cNorm_stdev, orientation='horizontal')
cb.ax.tick_params(axis='x', labelsize=13)
cb.ax.set_title('Uncertainty', y=1.35, fontsize=19)

for filetype in ['png', 'pdf']:
    p.savefig('HUEflux_LUMstdev_fullmap.{}'.format(filetype), dpi=300, bbox_inches='tight')

## 7. Show original map and Eigenspectra

In [ ]:
def show_orig_map(lam,spaxels,waveInd=0):#testNum=1):
    """
    Show the original map at a given wavelength
    
    Parameters
    -----------
    waveInd: int
        The wavelength index
    testNum: int
        The test Number (ie. lightcurve number)
    """
    #origData = np.load("data/maps/mystery_map{}.npz".format(testNum))
    lammin1 = 2.41; lammax1 = 3.98; dlam1 = 0.18
    #spaxels = origData["spaxels"]
    #lam = origData["wl"]
    lamlo, dlamlo = gen_lightcurves.construct_lam(lammin1, lammax1, dlam=dlam1)
    Nlamlo = len(lamlo)

    # Set HealPy pixel numbers
    Npix = spaxels.shape[0]

    # Define empty 2d array for spaxels
    spec2d = np.zeros((Npix, Nlamlo))

    # Loop over pixels filling with spectra
    for i in range(Npix):
        # Degrade the spectra to lower resolution
        spec2d[i,:] = gen_lightcurves.downbin_spec(spaxels[i, :], lam, lamlo, dlam = dlamlo)


    healpy.mollview(spec2d[:,waveInd], title=r"%0.2f $\mu$m" %lamlo[waveInd])
    p.show()
    return spec2d

In [ ]:
spec2D=show_orig_map(lamhr,spaxels,waveInd=3)
groupstarts=np.unique(spec2D[:,0])
p.figure()
for f in groupstarts:
    p.plot(waves,spec2D[spec2D[:,0]==f][0,:])
p.show()

# II.  A More Difficult Model

## 1. Generate lightcurve using STARRY

In [ ]:
model_name = 'secretmodel' #name of the model to perform the fit from

# Get the high-res wavelength grid and pixel spectra (spaxels)
lamhr, spaxels = gen_lightcurves.prep_map1()
spec2D=show_orig_map(lamhr,spaxels,waveInd=9)
groupstarts=np.unique(spec2D[:,0])
# p.figure()
# for f in groupstarts:
#     p.plot(waves,spec2D[spec2D[:,0]==f][0,:])
# plt.show()

# Generate the multi-wavelength eclipse lightcurves using starry
time, lam, dlam, lcurves = gen_lightcurves.create_lightcurves_with_starry(lamhr, spaxels, plot_lightcurves = True, plot_diagnostic = True)

# Save as a .npz file in data/input_lightcurves
filename='data/input_lightcurves/'+model_name+'.npz'
np.savez(filename,time=time,wl=lam,dwl=dlam,lightcurve=lcurves)

## 2. Fit eigencurves to lightcurve

In [ ]:
## Results are saved in data/sph_harmonic_coefficients_full_samples
for oneOrd in np.arange(5,6): #which degrees of spherical harmonics you want to create fits for
    run_higher_sph_harm.run_lc_noise_and_fit(norder=oneOrd,usePath=filename)

## 3. Display maps for median, 5th percentile, and 95th percentile from spherical harmonic fit

In [ ]:
dataDir='data/sph_harmonic_coefficients_full_samples/'+model_name+'/'
waves, lats, lons = get_map_and_plot(waveInd=9,degree=5,dataDir=dataDir) #FINDME: using a different waveInd will show maps at a different wavelength; using a different degree will show a fit with a different number of spherical harmonics


## 4. Use K means clustering to find Eigenspectra

In [ ]:
degree=5 #FINDME: change this to investigate fits with different numbers of spherical harmonics
ngroups=4 #FINDME: change ngroups to cluster the data into a different number of eigengroups
eigenspectra_draws, kgroup_draws, maps = find_groups(dataDir,degree=degree,trySamples=200,ngroups=ngroups)
#print("{}spherearray_deg_{}.npz".format(dataDir,degree))

## 5. Plot the Eigenspectra

In [ ]:
# THIS IS THE PART THAT NEEDS TO BE UPDATED FOR THE ERRORS THING
eigenspectra = np.mean(eigenspectra_draws, axis=0)
eigenerrs = np.std(eigenspectra_draws, axis=0)
kgroups = np.mean(kgroup_draws, axis=0)

counter=0
colors=['b','g','orange','red']
for spec, err in zip(eigenspectra, eigenerrs):
    p.errorbar(waves, spec, err,label=('Group '+np.str(counter)),linewidth=2,marker='.',markersize=10,color=colors[counter])
    counter+=1
p.xlabel('Wavelength (micron)',fontsize=20)
p.ylabel('Fp/Fs',fontsize=20)
p.tick_params(labelsize=20,axis="both",right=True,top=True,width=1.5,length=5)
p.title('Eigenspectra')
p.legend(fontsize=15)
p.show()

## 6. Plot Spatial Map of Eigengroups

In [ ]:
full_extent = np.array([np.min(lons),np.max(lons),np.min(lats),np.max(lats)])/np.pi*180
# londim, latdim = np.shape(maps)[1:]

maps_mean = np.average(maps, axis=0)
maps_error = np.std(maps, axis=0)

cmap = cc.cm['isolum']
cmap_grey = cc.cm['linear_grey_10_95_c0']
# norm = Normalize(vmin=np.min(maps_mean), vmax=np.max(maps_mean))

In [ ]:
p.figure(figsize=(10,6.5))
p.title('Eigengroups', fontsize=22)


group_map = generate_map2d(hue_quantity=0.5*kgroups,
                           lightness_quantity=maps_mean,
                           hue_cmap=cmap,
                           scale_min=10,
                           scale_max=90)
p.imshow(group_map, extent=full_extent, interpolation='gaussian')
CS = p.contour(lons/np.pi*180, -lats/np.pi*180, 2*kgroups,
               levels=np.arange(ngroups), colors='k', linestyles=['solid', 'dashed', 'dotted'])

p.clabel(CS, inline=1, fmt='%1.0f', fontsize=12)

p.xlabel(r'Longitude ($^\circ$)', fontsize=16)
p.ylabel(r'Latitude ($^\circ$)', fontsize=16)
p.setp(p.axes().get_xticklabels(), fontsize=16)
p.setp(p.axes().get_yticklabels(), fontsize=16)

cmap_group = cmap
cNorm_group  = Normalize(vmin=0, vmax=ngroups-1)
scalarMap_group = cm.ScalarMappable(norm=cNorm_group, cmap=cmap_group)

cmap_flux = cmap_grey
cNorm_flux  = Normalize(vmin=0, vmax=np.nanmax(maps_mean))
scalarMap_flux = cm.ScalarMappable(norm=cNorm_flux, cmap=cmap_flux)

bounds = np.linspace(-0.5, ngroups-0.5, ngroups+1)
norm_group = BoundaryNorm(bounds, cmap_group.N)

divider = make_axes_locatable(p.axes())
ax2 = divider.append_axes("bottom", size="7.5%", pad=1)
cb = colorbar.ColorbarBase(ax2, cmap=cmap_group, norm=norm_group, spacing="proportional", orientation='horizontal', ticks=np.arange(0, ngroups, 1), boundaries=bounds)
cb.ax.xaxis.set_minor_formatter(FormatStrFormatter('%.1g'))
cb.ax.tick_params(axis='x', labelsize=13)
cb.ax.tick_params(axis='x', direction='inout',  top='off', bottom='off',
                  labeltop='on', labelbottom='off', labelsize=13, pad=-15)
cb.ax.set_title('Group', y=1.35, fontsize=19)

ax3 = divider.append_axes("bottom", size="7.5%", pad=0.75)
cb = colorbar.ColorbarBase(ax3, cmap=cmap_flux, norm=cNorm_flux, orientation='horizontal')
cb.ax.tick_params(axis='x', labelsize=13)
cb.ax.set_title('Flux', y=1.35, fontsize=19)

for filetype in ['png', 'pdf']:
    p.savefig('HUEgroup_LUMflux_complexmap.{}'.format(filetype), dpi=300, bbox_inches='tight')

In [ ]:
p.figure(figsize=(10,6.5))
p.title('Flux', fontsize=22)

group_map = generate_map2d(hue_quantity=(maps_mean-np.min(maps_mean))/np.ptp(maps_mean),
                           lightness_quantity=1-((maps_error-np.min(maps_error))/np.ptp(maps_error)),
                           hue_cmap=cmap,
                           scale_min=10,
                           scale_max=90)
p.imshow(group_map, extent=full_extent, interpolation='gaussian')
CS = p.contour(lons/np.pi*180, -lats/np.pi*180, 2*kgroups,
               levels=np.arange(ngroups), colors='k', linestyles=['solid', 'dashed', 'dotted'])

p.clabel(CS, inline=1, fmt='%1.0f', fontsize=12)

p.xlabel(r'Longitude ($^\circ$)', fontsize=16)
p.ylabel(r'Latitude ($^\circ$)', fontsize=16)
p.setp(p.axes().get_xticklabels(), fontsize=16)
p.setp(p.axes().get_yticklabels(), fontsize=16)

cmap_flux = cmap
cNorm_flux = Normalize(vmin=0, vmax=np.nanmax(maps_mean))
scalarMap_flux = cm.ScalarMappable(norm=cNorm_flux, cmap=cmap_flux)

cmap_stdev = cmap_grey
cNorm_stdev  = Normalize(vmin=0, vmax=np.nanmax(maps_error))
scalarMap_stdev = cm.ScalarMappable(norm=cNorm_stdev, cmap=cmap_stdev)

divider = make_axes_locatable(p.axes())
ax2 = divider.append_axes("bottom", size="7.5%", pad=1)
cb = colorbar.ColorbarBase(ax2, cmap=cmap_flux, norm=cNorm_flux, orientation='horizontal')
cb.ax.tick_params(axis='x', labelsize=13)
cb.ax.set_title('Flux', y=1.35, fontsize=19)

ax3 = divider.append_axes("bottom", size="7.5%", pad=0.75)
cb = colorbar.ColorbarBase(ax3, cmap=cmap_stdev, norm=cNorm_stdev, orientation='horizontal')
cb.ax.tick_params(axis='x', labelsize=13)
cb.ax.set_title('Uncertainty', y=1.35, fontsize=19)

for filetype in ['png', 'pdf']:
    p.savefig('HUEflux_LUMstdev_complexmap.{}'.format(filetype), dpi=300, bbox_inches='tight')

## 7. Show original map and Eigenspectra

In [ ]:
spec2D=show_orig_map(lamhr,spaxels,waveInd=3)
groupstarts=np.unique(spec2D[:,0])
p.figure()
for f in groupstarts:
    p.plot(waves,spec2D[spec2D[:,0]==f][0,:])
p.show()

# Possible Sections to Keep/Remove

## Display Eigenspectra from posterior draws individually (instead of just the means)

In [ ]:
#Do we need this section or is it just for testing?
p.figure()
for oneSpec in np.array(eigenspectra_draws):
    for deg in np.arange(ngroups):
        p.plot(waves,oneSpec[deg,:],color='b',alpha=0.05)
p.xlabel('Wavelength ($\mu$m)')
p.ylabel('F$_p$/F$_*$')

## Loop over many samples to get eigenmap and eigenspectra Errors

In [ ]:
def show_eigenspec_and_map(ngroups=4,degree=5):
    """ Show the eigenspectra and map"""
    fig, (ax0, ax1) = p.subplots(1,2,figsize=(12,4))
    for ind,spec, err in zip(range(ngroups), eigenspectra, eigenerrs):
        ax0.errorbar(waves, spec, err, marker='o',
                  color=p.cm.viridis(float(ind)/float(ngroups-1)))
    ax0.set_xlabel('Wavelength (micron)')
    ax0.set_ylabel('Fp/Fs')
    ax0.set_title('Eigenspectra from light-curve fit, n={}'.format(degree))

    imData = ax1.imshow(kgroups)
    cbar = fig.colorbar(imData, ticks=np.arange(ngroups),ax=ax1)
    cbar.set_label('# Group')
    ax1.set_xlabel('Latitude')
    ax1.set_ylabel('Longitude')
    ax1.set_title('Retrieved group map')

In [ ]:
show_eigenspec_and_map()